In [ ]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")

# 사용자 코딩
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 2000)
X_id = X_test['cust_id']

# 결측값 보완
X_temp = pd.concat([X_train, X_test], axis=0).reset_index(drop=True)
X_temp.fillna(0, inplace=True)

# 이상치 제거
# 'cust_id', '총구매액', '최대구매액', '환불금액', '주구매상품', '주구매지점', '내점일수', '내점당구매건수', '주말방문비율', '구매주기'
print(X_temp.columns)
items = ['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']
for col in items:
	print(col, " 항목 조정 !!")
	quan25 = X_temp[col].quantile(0.25)
	quan75 = X_temp[col].quantile(0.75)
	quanDiff = (quan75 - quan25) * 1.5
	mask_min = (X_temp[col] < quan25 - quanDiff)
	mask_max = (X_temp[col] > quan25 + quanDiff) 
	X_temp.loc[mask_min, col] = quan25 - quanDiff
	X_temp.loc[mask_max, col] = quan25 + quanDiff
	
# cat, num 분리
X_cat = X_temp[['주구매상품', '주구매지점']]
X_num = X_temp[['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']]

# 더미변수화
X_cat_dummy = pd.get_dummies(X_cat)

# 정규화(스케일링)
from sklearn.preprocessing import MinMaxScaler
X_num_scaled = MinMaxScaler().fit_transform(X_num)
X_num_scaled = pd.DataFrame(X_num_scaled, index=X_num.index, columns=X_num.columns)

# 변수 통합 및 train, test 분리
X_temp = pd.concat([X_num_scaled, X_cat_dummy], axis=1)
X_train = X_temp[:3500]
X_test = X_temp[3500:]
y_train = y_train['gender']


# 모델 적용 Stacking vs XGBoost
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier

estimators = [('sv', SVC()), ('rf', RandomForestClassifier(max_features='sqrt'))]
model_stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression()).fit(X_train, y_train)
# model_xgb = XGBClassifier().fit(X_train, y_train)

# cross validation score
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# kfold = KFold(n_splits=3, random_state=3456, shuffle=True)
# score = cross_val_score(StackingClassifier(estimators=estimators, final_estimator=LogisticRegression()), X_train, y_train, cv=kfold)
# score = cross_val_score(XGBClassifier(), X_train, y_train, cv=kfold)
# score = cross_val_score(LogisticRegression(), X_train, y_train, cv=kfold)
# print("교차검증 결과 : ", score)
# print("교차검증 결과 : ", score.mean())

# StackingClassifier - 미정제 데이터가 많을 수록 유리.
# 교차검증 결과 :  [0.63496144 0.65552699 0.66209262]
# 교차검증 결과 :  0.650860352077794
# 이상치 미정재시 : 65

# XGBClassifier
# 교차검증 결과 :  [0.61525278 0.63067695 0.63036021]
# 교차검증 결과 :  0.625429980064505
# 이상치 미정재시 : 62

# 로지스틱 회귀모형 - 데이터 정제가 잘되어 있을수록 단순 성능이 더 좋다.
# 교차검증 결과 :  [0.64610111 0.66238218 0.68782161]
# 교차검증 결과 :  0.6654349676127821
# 이상치 미정재시 : 64

# 테스트 데이터 적용
pred_test = model_stack.predict_proba(X_test)

result = pd.DataFrame({'cust_id': X_id, 'gender': pred_test[:, 1]})
print(result)
result.to_csv('003000000.csv', index=False)

import os.path
print(os.path.exists('003000000.csv'))

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)
